In [ ]:
# ✅ LOM Cushion LoRA Trainer (Colab 노트북)

# === [1] 환경설정 ===
# Colab 기본 torch==2.6.0, torchvision==0.21.0, torchaudio==2.6.0 유지
!pip install -U torch torchvision torchaudio  # Colab 기본 버전 업그레이드 유지
!pip install -q xformers==0.0.29.post1 diffusers==0.26.3 transformers==4.41.1 accelerate==0.30.1 safetensors==0.4.2 bitsandbytes==0.43.1 peft==0.10.0 huggingface_hub==0.25.1 gradio==4.24.0

from huggingface_hub import notebook_login
notebook_login()  # 🤗 Hugging Face 로그인 (토큰 필요)


In [ ]:
# === [2] Hugging Face Dataset에서 이미지 다운로드 ===
!git clone https://huggingface.co/datasets/sun2141/lom-cushion-images

from pathlib import Path
image_dir = Path("lom-cushion-images/images")


In [ ]:
# === [3] 데이터셋 준비 ===
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

caption_file = Path("lom-cushion-images/prompts/image_caption.csv")
captions_df = pd.read_csv(caption_file)

class CushionDataset(Dataset):
    def __init__(self, dataframe, image_dir):
        self.df = dataframe
        self.image_dir = image_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(self.image_dir / row["filename"]).convert("RGB").resize((512, 512))
        image_np = np.array(image)
        image_tensor = torch.tensor(image_np).permute(2, 0, 1).float() / 255.0
        return image_tensor, row["prompt"]

train_dataset = CushionDataset(captions_df, image_dir)
train_loader = DataLoader(train_dataset, batch_size=1)


In [ ]:
# === [4] 모델 준비 및 LoRA 학습 ===
from diffusers import StableDiffusionPipeline
from peft import LoraConfig, get_peft_model

base_model = "stabilityai/stable-diffusion-2-1-base"
pipeline = StableDiffusionPipeline.from_pretrained(base_model, torch_dtype=torch.float16)

lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=["attn2.to_q", "attn2.to_k", "attn2.to_v", "attn2.to_out.0"])
model = get_peft_model(pipeline.unet, lora_config)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(1):
    for images, prompts in train_loader:
        loss = model(images).loss  # 실제로는 LoRA용 loss 함수 필요 (단순화된 예시)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [ ]:
# === [5] Hugging Face 업로드 ===
model.save_pretrained("./lora-cushion")
!huggingface-cli upload ./lora-cushion "sun2141/lora-cushion" --repo-type model


In [ ]:
# === [6] Gradio 이미지 생성 ===
import gradio as gr

def generate(prompt):
    pipe = StableDiffusionPipeline.from_pretrained("sun2141/lora-cushion", torch_dtype=torch.float16)
    return pipe(prompt).images[0]

gr.Interface(fn=generate, inputs="text", outputs="image").launch(share=True)
